# Unit-тестирование


## Простое тестирование

`assert` - проверяет условие, если условие неверно - выдает ошибку `AssertionError`:

In [ ]:
nums = [1, 2, 4]

assert sum(nums) == 6, "Sum should be 6"

## Модуль unittest

В Python встроен модуль **unittest**, который поддерживает автоматизацию тестов, использование общего кода для настройки и завершения тестов, объединение тестов в группы, а также позволяет отделять тесты от фреймворка для вывода информации.

Для автоматизации тестов, unittest поддерживает некоторые важные концепции:
- **Испытательный стенд (test fixture)** - выполняется подготовка, необходимая для выполнения тестов и все необходимые действия для очистки после выполнения тестов. Это может включать, например, создание временных баз данных или запуск серверного процесса.

- **Тестовый случай (test case)** - минимальный блок тестирования. Он проверяет ответы для разных наборов данных. Модуль unittest предоставляет базовый класс TestCase, который можно использовать для создания новых тестовых случаев.

- **Набор тестов (test suite)** - несколько тестовых случаев, наборов тестов или и того и другого. Он используется для объединения тестов, которые должны быть выполнены вместе.

- **Исполнитель тестов (test runner)** - компонент, который управляет выполнением тестов и предоставляет пользователю результат. Исполнитель может использовать графический или текстовый интерфейс или возвращать специальное значение, которое сообщает о результатах выполнения тестов.


Простой пример проверки работы строковых методов:

In [1]:
import unittest

class TestStringMethods(unittest.TestCase):
    # тестовые методы должны начинаться с test
    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # Проверим, что s.split не работает, если разделитель - не строка
        with self.assertRaises(TypeError):
            s.split(2)

if __name__ == '__main__':

    unittest.main()

E
ERROR: /root/ (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/root/'

----------------------------------------------------------------------
Ran 1 test in 0.014s

FAILED (errors=1)


SystemExit: True

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### Опаньки...

А что случилось?

>The reason is that `unittest.main` looks at **sys.argv** and first parameter is what started IPython or Jupyter, therefore the error about kernel connection file not being a valid attribute. Passing explicit list to `unittest.main` will prevent IPython and Jupyter look at **sys.argv**. Passing **exit=False** will prevent unittest.main to shutdown the kernell process

Сохраним имеющийся код тестирования в файл (понадобится позже):

In [2]:
def dump_to(path):
    with open(path, 'w') as f:
        f.write(_i)  # _i это "последний выполненный Input" в iPython

dump_to('strings.py')

Cкорректируем код наших тестов:

In [ ]:
class TestStringMethods(unittest.TestCase):
    # тестовые методы должны начинаться с test
    def test_upper(self):
        self.assertEqual('foo'.upper(), 'FOO')

    def test_isupper(self):
        self.assertTrue('FOO'.isupper())
        self.assertFalse('Foo'.isupper())

    def test_split(self):
        s = 'hello world'
        self.assertEqual(s.split(), ['hello', 'world'])
        # Проверим, что s.split не работает, если разделитель - не строка
        with self.assertRaises(TypeError):
            s.split(2)

if __name__ == '__main__':

    unittest.main(argv=['first-arg-is-ignored'], exit=False)

## Интерфейс командной строки

Можем тестировать отдельный модуль, класс или метод:

In [3]:
!python3 -m unittest strings                               # модуль
!python3 -m unittest strings.TestStringMethods             # класс
!python3 -m unittest strings.TestStringMethods.test_split  # метод

...
----------------------------------------------------------------------
Ran 3 tests in 0.000s

OK
...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK
.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


С помощью флага *-v* можно получить более детальный отчёт

In [4]:
#:
!python3 -m unittest -v strings

test_isupper (strings.TestStringMethods) ... ok
test_split (strings.TestStringMethods) ... ok
test_upper (strings.TestStringMethods) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


Ещё флаги:
- **-b (--buffer)** - вывод программы при провале теста будет показан, а не скрыт, как обычно.
- **-c (--catch)** - Ctrl+C ожидает завершения текущего теста и сообщает текущие результаты, второе нажатие - обычное поведение.
- **-f (--failfast)** - выход после первого же неудачного теста.
- **--locals** (начиная с Python 3.5) - показывать локальные переменные для провалившихся тестов.

## Обнаружение тестов

`unittest` поддерживает простое обнаружение тестов. Для совместимости с обнаружением тестов все файлы тестов должны быть модулями или пакетами, импортируемыми из директории верхнего уровня проекта ([см. подробнее о правилах наименования модулей](https://pythonworld.ru/osnovy/rabota-s-modulyami-sozdanie-podklyuchenie-instrukciyami-import-i-from.html#id3)).

Обнаружение тестов реализовано в `TestLoader.discover()`, но может быть использовано из командной строки:

In [ ]:
!mv strings.py test_strings.py  #чтобы сработало переименуем модуль в test....py
!python3 -m unittest discover

## Организация тестового кода

Создадим класс, который будем тестировать:

In [5]:
class Widget():
    def __init__(self, name, x = 50, y = 50):
        self.name = name
        self.x = x
        self.y = y

    def size(self):
        return (self.x, self.y)

    def resize(self, size):
        (self.x, self.y) = size

Базовые блоки тестирования это **тестовые случаи** - простые случаи, которые должны быть проверены на корректность.

Тестовый случай создаётся путём наследования от unittest.TestCase.

Тестирующий код должен быть самостоятельным, то есть никак не зависеть от других тестов.

Простейший подкласс **TestCase** может просто реализовывать тестовый метод (метод, начинающийся с test)

In [ ]:
class DefaultWidgetSizeTestCase(unittest.TestCase):
    def test_default_widget_size(self):
        widget = Widget('The widget')
        self.assertEqual(widget.size(), (50, 50))

if __name__ == '__main__':

    unittest.main(argv=['',], defaultTest='DefaultWidgetSizeTestCase', exit=False)

Тестов может быть много, и часть кода настройки может повторяться. К счастью, мы можем определить код настройки путём реализации метода `setUp()`, который будет запускаться **перед каждым тестом**.

Мы также можем определить метод `tearDown()`, который будет запускаться **после каждого теста**.

In [ ]:
class SimpleWidgetTestCase(unittest.TestCase):
    def setUp(self):
        #когда:
        self.widget = Widget('The widget')

    def test_default_widget_size(self):
        self.assertEqual(self.widget.size(), (50,50),
                         'incorrect default size')

    def test_widget_resize(self):
        #при действии:
        self.widget.resize(100,150)
        #происходит...
        self.assertEqual(self.widget.size(), (100,150),
                         'wrong size after resize')

    def test(self):
        #когда:

        #при действии:

        #происходит...




    def tearDown(self):
        pass

if __name__ == '__main__':
    unittest.main(argv=['',], defaultTest='SimpleWidgetTestCase', exit=False)

Можно разместить все тесты в том же файле, что и сама программа (таком как *widgets.py*), но размещение тестов в отдельном файле (таком как *test_widget.py*) имеет много преимуществ:

- Модуль с тестом может быть запущен автономно из командной строки.
- Тестовый код может быть легко отделён от программы.
- Меньше искушения изменить тесты для соответствия коду программы без видимой причины.
- Тестовый код должен изменяться гораздо реже, чем программа.
- Протестированный код может быть легче переработан.
- Тесты для модулей на C должны быть в отдельных модулях, так почему же не быть последовательным?
- Если стратегия тестирования изменяется, нет необходимости изменения кода программы.

## Пропуск тестов и ожидаемые ошибки

`unittest` поддерживает пропуск отдельных тестов, а также классов тестов. Вдобавок, поддерживается пометка теста как *"не работает, но так и надо"*.

Пропуск теста осуществляется использованием декоратора `skip()` или одного из его условных вариантов:

In [ ]:
__version__ = (0, 9)
platform = "ubuntu"


class MyTestCase(unittest.TestCase):
    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(__version__ < (1, 3), "not supported in this library version")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

    @unittest.skipUnless(platform.startswith("win"), "requires Windows")
    def test_windows_support(self):
        # windows specific testing code
        pass

if __name__ == '__main__':
    unittest.main(argv=['','-v'], defaultTest='MyTestCase', exit=False)

Классы также могут быть пропущены:

In [ ]:
@unittest.skip("showing class skipping")
class MySkippedTestCase(unittest.TestCase):
    def test_not_run(self):
        pass

if __name__ == '__main__':

    unittest.main(argv=['','-v'], defaultTest='MySkippedTestCase', exit=False)

Тесты, в которых ожидаются ошибки, используют декоратор `expectedFailure()`:

In [ ]:
class ExpectedFailureTestCase(unittest.TestCase):
    @unittest.expectedFailure
    def test_fail(self):
        self.assertEqual(1, 0, "broken")

if __name__ == '__main__':
    unittest.main(argv=['','-v'], defaultTest='ExpectedFailureTestCase', exit=False)

Очень просто сделать свой декоратор. Например, следующий декоратор пропускает тест, если переданный объект не имеет указанного атрибута:

In [ ]:
obj1 = [1, 2, 3]

def skipUnlessHasattr(obj, attr):
    if hasattr(obj, attr):
        return lambda func: func
    return unittest.skip(f"{obj} doesn't have {attr}")

class YetAnotherTestCase(unittest.TestCase):
    @skipUnlessHasattr(obj1,'add')
    def test_fail(self):
        pass

if __name__ == '__main__':
    unittest.main(argv=['','-v'], defaultTest='YetAnotherTestCase', exit=False)

**Замечание** - для пропущенных тестов не запускаются `setUp()` и `tearDown()`, для пропущенных классов не запускаются `setUpClass()` и `tearDownClass()`, для пропущенных модулей не запускаются `setUpModule()` и `tearDownModule()`.

Эй, а что ещё за `setUpClass()` и `setUpModule()`?

In [ ]:
import unittest

class Test(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls._connection = createExpensiveConnectionObject()

    @classmethod
    def tearDownClass(cls):
        cls._connection.destroy()


#These should be implemented as functions:
def setUpModule():
    createConnection()

def tearDownModule():
    closeConnection()

del setUpModule
del tearDownModule

## Различение итераций теста с помощью подтестов

Когда некоторые тесты имеют лишь незначительные отличия, например некоторые параметры, `unittest` позволяет различать их внутри одного тестового метода, используя менеджер контекста `subTest()`:

In [ ]:
class NumbersTest(unittest.TestCase):
    def test_even(self):
        """Test that numbers between 0 and 3 are all even"""
        for i in range(0, 4):
            with self.subTest(i=i):
                self.assertEqual(i % 2, 0)

unittest.main(argv=['','-v'], defaultTest='NumbersTest', exit=False)

Можем кастомизировать запуск имеющихся тестов:

In [ ]:
def MySuite():
    suite = unittest.TestSuite()
    suite.addTest(SimpleWidgetTestCase('test_default_widget_size'))
    suite.addTest(SimpleWidgetTestCase('test_widget_resize'))
    suite.addTest(NumbersTest('test_even'))
    suite.addTest(YetAnotherTestCase('test_fail'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(MySuite())

## Проверки на успешность

Модуль unittest предоставляет множество функций для самых различных проверок:

- `assertEqual(a, b)` — `a == b`
- `assertNotEqual(a, b)` — `a != b`
- `assertTrue(x)` — `bool(x) is True`
- `assertFalse(x)` — `bool(x) is False`
- `assertIs(a, b)` — `a is b`
- `assertIsNot(a, b)` — `a is not b`
- `assertIsNone(x)` — `x is None`
- `assertIsNotNone(x)` — `x is not None`
- `assertIn(a, b)` — `a in b`
- `assertNotIn(a, b)` — `a not in b`
- `assertCountEqual(a, b)` — `a` и `b` содержат те же элементы в одинаковых количествах, но порядок не важен

- `assertIsInstance(a, b)` — `isinstance(a, b)`
- `assertNotIsInstance(a, b)` — `not isinstance(a, b)`
- `assertRaises(exc, fun, *args, **kwds)` — `fun(*args, **kwds)` порождает исключение exc
- `assertRaisesRegex(exc, r, fun, *args, **kwds)` — `fun(*args, **kwds)` порождает исключение `exc` и сообщение соответствует регулярному выражению `r`
- `assertWarns(warn, fun, *args, **kwds)` — `fun(*args, **kwds)` порождает предупреждение
- `assertWarnsRegex(warn, r, fun, *args, **kwds)` — `fun(*args, **kwds)` порождает предупреждение и сообщение соответствует регулярному выражению `r`
- `assertAlmostEqual(a, b)` — `round(a-b, 7) == 0`
- `assertNotAlmostEqual(a, b)` — `round(a-b, 7) != 0`
- `assertGreater(a, b)` — `a > b`
- `assertGreaterEqual(a, b)` — `a >= b`
- `assertLess(a, b)` — `a < b`
- `assertLessEqual(a, b)` — `a <= b`
- `assertRegex(s, r)` — `r.search(s)`
- `assertNotRegex(s, r)` — `not r.search(s)`